# Very primitive particle-particle interaction in Peano 4

A very simple demonstration how to extend the simple particle setup with an explicit Euler and some simple particle-particle interaction. We use a fake potential, so there's no physical meaning in this example. However, it demonstrates the principles behind multibody simulations with the Peano toolbox.

In [1]:
import os

import peano4
import peano4.dastgen2
import peano4.toolbox
import peano4.toolbox.particles
import dastgen2


DaStGen 2 (C) www.peano-framework.org
Peano 4 (C) www.peano-framework.org


In [2]:
project = peano4.Project( ["examples", "particles"], "particle-particle-interaction", "." )

In [3]:
build_mode = peano4.output.CompileMode.Asserts
project.output.makefile.parse_configure_script_outcome( "../../.." )
project.output.makefile.set_mode( build_mode )

parse configure outcome ../../../config.log to extract configure settings
found the configure call info   $ ./configure CXX=icpx --enable-exahype --enable-loadbalancing --with-multithreading=omp CXXFLAGS=--std=c++17 -fopenmp -DnoMPISupportsSingleSidedCommunication LDFLAGS=-fopenmp --enable-blockstructured --enable-particles

parse configure outcome ../../../src/Makefile to extract compile settings
add CXX=icpx
add CXXFLAGS=--std=c++17 -fopenmp -DnoMPISupportsSingleSidedCommunication
add CXXFLAGS_PEANO_2D_ASSERTS=-DDimensions=2 -DPeanoDebug=2 -g3 -O0
add CXXFLAGS_PEANO_2D_DEBUG=-DDimensions=2 -DPeanoDebug=4 -g3 -O0
add CXXFLAGS_PEANO_2D_RELEASE=-DDimensions=2 -DPeanoDebug=0
add CXXFLAGS_PEANO_2D_STATS=-DDimensions=2 -DPeanoDebug=0 -DTrackStatistics
add CXXFLAGS_PEANO_2D_TRACE=-DDimensions=2 -DPeanoDebug=1 -g3
add CXXFLAGS_PEANO_3D_ASSERTS=-DDimensions=3 -DPeanoDebug=2 -g3 -O0
add CXXFLAGS_PEANO_3D_DEBUG=-DDimensions=3 -DPeanoDebug=4 -g3 -O0
add CXXFLAGS_PEANO_3D_RELEASE=-DDimensions=3 -

## Model the particle

This time, we add an additional velocity field to the particle. We use Peano's double array here for the velocities to have full support of Peano's vector classes. Instead of saying that this array had two or three entries, we parameterise it through the symbol Dimensions. This way, our code will immediately run for 2d and 3d without any changes. Dimensions is a constant that Peano 4 defines. There are few of these, but Dimensions is really the important one.

In [4]:
particle  = peano4.toolbox.particles.Particle( "Particle" )
particle.data.add_attribute( peano4.dastgen2.Peano4DoubleArray("v","Dimensions") )
particles = peano4.toolbox.particles.ParticleSet( particle )

In [5]:
project.datamodel.add_global_object(particle)
project.datamodel.add_vertex(particles)

## Model the setup phases

The setup remains almost unchanged. The one tiny little difference is that we initialise the velocity properly when we create a particle. The generator above gives you different settings for v. We set the velocity component-wisely here.

In [6]:
mesh_size = 0.2

create_grid = peano4.solversteps.Step( name="CreateGrid", add_user_defined_actions=False )
create_grid.add_action_set( peano4.toolbox.CreateRegularGrid(mesh_size) )
create_grid.use_vertex(particles)
create_grid.use_cell(particle_tree_analysis.cell_marker)
project.solversteps.add_step(create_grid)

NameError: name 'particle_tree_analysis' is not defined

In [7]:
particle_tree_analysis = peano4.toolbox.particles.ParticleTreeAnalysis(particles)
project.datamodel.add_cell(particle_tree_analysis.cell_marker)   # read docu of ParticleTreeAnalysis

initialisation_snippet = """
 // we could do something intelligent here, but we actually
 // just set the cut-off radius
 particle->setCutOffRadius(0.001);
 particle->setV(0,0.0);
 particle->setV(1,0.0);
"""

init_setup = peano4.solversteps.Step( name="Init", add_user_defined_actions=False )
init_setup.use_vertex(particles)
init_setup.use_cell(particle_tree_analysis.cell_marker)
init_setup.add_action_set( peano4.toolbox.particles.UpdateParticleGridAssociation(particles) )
init_setup.add_action_set( peano4.toolbox.particles.InsertRandomParticlesIntoUnrefinedCells(
  particle_set=particles,
  average_distance_between_particles=mesh_size/10,
  initialisation_call=initialisation_snippet,
  noise=True ))
init_setup.add_action_set( peano4.toolbox.particles.ParticleAMR(particles,particle_tree_analysis) )
init_setup.add_action_set( particle_tree_analysis )
project.solversteps.add_step(init_setup)

In [8]:
print_solution = peano4.solversteps.Step( "Plot", add_user_defined_actions=False )
print_solution.use_vertex(particles)
print_solution.use_cell(particle_tree_analysis.cell_marker)
print_solution.add_action_set( peano4.toolbox.PlotGridInPeanoBlockFormat( filename="grid", time_stamp_evaluation=peano4.toolbox.PlotGridInPeanoBlockFormat.CountTimeSteps ) )
print_solution.add_action_set( peano4.toolbox.particles.PlotParticlesInVTKFormat( "particles", particles, time_stamp_evaluation=peano4.toolbox.particles.PlotParticlesInVTKFormat.CountTimeSteps ) )
project.solversteps.add_step(print_solution)


## Particle behaviour

Our particle behaviour corresponds to a new algorithmic phase: the move. Compared to the other algorithmic steps, the big (technical) difference this time is that we ask Peano to give us a user-defined hook-in point for the action set. That is, not all the behaviour is realised via pre-manufactured action sets anymore.

In [9]:
move_particles = peano4.solversteps.Step( "Move" )
move_particles.use_vertex(particles)
move_particles.use_cell(particle_tree_analysis.cell_marker)
move_particles.add_action_set( peano4.toolbox.particles.UpdateParticleGridAssociation(particles) )
move_particles.add_action_set( peano4.toolbox.particles.ParticleAMR(particles,particle_tree_analysis) )
move_particles.add_action_set( particle_tree_analysis )
project.solversteps.add_step(move_particles)

## Generate the actual C++ code

Next, we generate the actual Peano 4 C++ code:

In [10]:
project.generate()

generate all code ...
user has to modify class Move in actions directory manually 
generated particle-particle-interaction-main.cpp
write ./Makefile
./vertexdata/ParticleSet.h written by Jinja2TemplatedHeaderImplementationFilePair.py (from template /home/tobias/git/Peano/python/peano4/toolbox/particles/ParticleSet.template.h)
./vertexdata/ParticleSet.cpp written by Jinja2TemplatedHeaderImplementationFilePair.py (from template /home/tobias/git/Peano/python/peano4/toolbox/particles/ParticleSet.template.cpp)
./repositories/DataRepository.h written by TemplatedHeaderImplementationFilePair.py
./repositories/DataRepository.cpp written by TemplatedHeaderImplementationFilePair.py
././observers/Init2peano4_toolbox_particles_UpdateParticleGridAssociation0.h written by ActionSet.py
././observers/Init2peano4_toolbox_particles_UpdateParticleGridAssociation0.cpp written by ActionSet.py
././observers/Init2peano4_toolbox_particles_InsertRandomParticlesIntoUnrefinedCells1.h written by ActionSet.py
././

## Write the main

There is one file that the Python API cannot write for you: That's the main file. In the present case, the main file is close to trivial: 


    int gridConstructionSteps = 0;
    while (peano4::parallel::SpacetreeSet::getInstance().getGridStatistics().getStationarySweeps()<5) {
      logInfo( "main()", "run a grid construction step and insert particles" );
      examples::particles::observers::CreateGrid createObserver;
      peano4::parallel::SpacetreeSet::getInstance().traverse(createObserver);
      gridConstructionSteps++;
    }

    logInfo( "main()", "finished grid construction after " << gridConstructionSteps << " steps, start timestepping" )

    examples::particles::observers::Init initObserver;
    peano4::parallel::SpacetreeSet::getInstance().traverse(initObserver);
    logInfo( "main()", "added particles" )

    examples::particles::observers::Plot plotObserver;
    peano4::parallel::SpacetreeSet::getInstance().traverse(plotObserver);
    logInfo( "main()", "dumped initial condition" )

We don't have to write the surrounding code ourself: If we run generate() as done above, we'll get a main file which we can use as a starting point. Once we have modified this blueprint, subsequent generate() calls won't overwrite our main files anymore. So we are on the save side.


## Build the code

With all in place, we can invoke the build. This could be a plain command line call to make (under the hood, Peano's Python API generates makefiles), but we can also use the Python API.

In [ ]:
project.build()

## Run code

Before we run any code, we clean up, i.e. remove all old output files.

In [ ]:
output_files = [ f for f in os.listdir(".") if f.endswith(".peano-patch-file") or f.endswith(".vtu") or f.endswith(".pvd") ]
for f in output_files:
  os.remove(f)


I could now run the code via a Python command

    #success = project.run( args=[], prefix=["mpirun", "-n", "1"] )
    
but instead I here decide to access the terminal directly via the notebook. Before you invoke the run, some remarks:

- Peano can build code in different versions. By default, I use the assert mode here which is excellent to find errors in the code. For production runs, you should use the release mode (or tracing if you wanna profile).
- Our code is not yet MPI-ready. Shared memory would work out of the box, but is not active as we have not yet provided a load balancing criterion. MPI requires slightly more work. So keep you problem smallish.
- If you translate in release mode, you'll get a lot of output. Tweak the filters in the main (or use a filter file) to study only those output you are actually interested in.

In [ ]:
# success = project.run( args=["--threads", "1"], prefix=["mpirun", "-n", "1"] )
!./peano4

## Visualisation

Peano 4 and its toolboxes provide multiple scripts to postprocess and visualise all output. You can directly visualise runtimes or load distributions with matplotlib, e.g. We don't go down this route in this simple example. We focus only on the particle and grid data.

The particle data has been written in a mainstream formats (vtk) that we can load directly into Paraview. This is fine for the time being. For complex simulations, you will want to use tailored, Peano 4-specific formats instead.

In [ ]:
from IPython.display import Image
!ls ../../python/peano4/toolbox/particles
Image("particles.png")

The grid is written in Peano's block format which relatively efficient. However, this format cannot be processed by Paraview directly. You first have to convert it. The guidebook discusses various ways to do this, but the Paraview's Python command is perhaps the simplest way:

In [ ]:
!pvpython ../../../python/peano4/visualisation/render.py --filter-fine-grid grid.peano-patch-file

We can now combine the particle visualisation with the actual grid. We see exactly what we would expect: A regular grid with roughly the same number of particles (10x10) per cell.

In [ ]:
from IPython.display import Image
!ls ../../python/peano4/toolbox/particles
Image("particles-with-grid.png")